<a href="https://colab.research.google.com/github/anas1IA/2024_new_package/blob/main/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-05 21:33:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2024-06-05 21:33:56 (83.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
# print(chars)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


## There are Many Possible Encodings

In Natural Language Processing (NLP), encoding refers to the various methods and architectures used to represent and process input sequences (like text) into a format suitable for the machine learning model to understand and work with. Below are the different types of encoding techniques and architectures:

### Observation on Vocabulary Size and Sequence Length in NLP:

- **Very small vocabularies lead to very long sequences of integers.**
- **Very large vocabularies lead to very short sequences of integers.**

#### Detailed Observations:

- **Very small vocabularies:** When the vocabulary size is small, each word in the text has to be represented by multiple tokens, resulting in very long sequences of integers. This is because many words will be broken down into smaller subwords or characters to fit within the limited vocabulary.
  - **Example:**
    - Vocabulary: {'a': 1, 'b': 2, 'c': 3, ...}
    - Sentence: "Hello world"
    - Encoded Sequence: [8, 5, 12, 12, 15, 27, 23, 15, 18, 12, 4] (assuming each character is a separate token)

- **Very large vocabularies:** When the vocabulary size is large, each word in the text can be represented by a single token, resulting in very short sequences of integers. This is because the extensive vocabulary can cover a wide range of words directly, reducing the need for subword tokenization.
  - **Example:**
    - Vocabulary: {'hello': 1, 'world': 2, ...}
    - Sentence: "Hello world"
    - Encoded Sequence: [1, 2] (assuming each word is a separate token)

### 1. **Bag of Words (BoW)**
- **Description:** Represents text as an unordered collection of words, ignoring grammar and word order. Each word in the text is mapped to a unique integer ID, and the text is represented by a vector counting the occurrences of each word.
- **Example:**
  - Text: "I love NLP and I love Python."
  - Vocabulary: {"I": 1, "love": 2, "NLP": 3, "and": 4, "Python": 5}
  - BoW Vector: [2, 2, 1, 1, 1] (Counts of each word in the vocabulary)

### 2. **Term Frequency-Inverse Document Frequency (TF-IDF)**
- **Description:** Extends BoW by assigning weights to words based on their frequency in a document relative to their frequency in the entire corpus.
- **Example:**
  - Text: "I love NLP and I love Python."
  - Corpus: ["I love NLP", "I love Python", "I love programming"]
  - TF-IDF for "I love NLP and I love Python":
    - "I": TF=2/6, IDF=log(3/3) = 0
    - "love": TF=2/6, IDF=log(3/3) = 0
    - "NLP": TF=1/6, IDF=log(3/1) = 1.1
    - "and": TF=1/6, IDF=log(3/1) = 1.1
    - "Python": TF=1/6, IDF=log(3/1) = 1.1
  - TF-IDF Vector: [0, 0, 0.18, 0.18, 0.18]

### 3. **Word Embeddings**
- **Word2Vec:**
  - **Description:** Uses neural networks to learn dense vector representations of words. There are two models: Continuous Bag of Words (CBOW) and Skip-gram.
  - **Example:**
    - "king" might be represented as a 300-dimensional vector: [0.2, 0.1, -0.3, ..., 0.5]
    - Similarity between "king" and "queen" can be calculated using cosine similarity.
- **GloVe (Global Vectors for Word Representation):**
  - **Description:** Uses word co-occurrence statistics to learn word vectors.
  - **Example:**
    - Co-occurrence matrix is built, and factorization produces vectors where "king" - "man" + "woman" ≈ "queen".
- **FastText:**
  - **Description:** Extends Word2Vec by representing words as bags of character n-grams, capturing subword information.
  - **Example:**
    - "running" might be broken into trigrams: ["run", "unn", "nni", "nin", "ing"], and each trigram has its own vector.

### 4. **Sentence and Document Embeddings**
- **Doc2Vec:**
  - **Description:** Extends Word2Vec to learn fixed-length vectors for variable-length texts like sentences, paragraphs, or documents.
  - **Example:**
    - Document: "I love NLP and Python."
    - Embedding: A 300-dimensional vector representing the entire document.
- **InferSent:**
  - **Description:** Pre-trained sentence embeddings using supervised learning with natural language inference (NLI) data.
  - **Example:**
    - Sentence: "The cat sat on the mat."
    - Embedding: A fixed-length vector (e.g., 4096 dimensions) capturing the meaning of the sentence.
- **Universal Sentence Encoder:**
  - **Description:** Provides embeddings for sentences and larger text blocks using Transformer or Deep Averaging Network (DAN) architectures.
  - **Example:**
    - Sentence: "NLP is fascinating."
    - Embedding: A 512-dimensional vector.

### 5. **Recurrent Neural Networks (RNNs)**
- **Description:** Processes sequences by maintaining a hidden state that captures information from previous time steps.
- **Example:**
  - Text: "I love NLP."
  - Each word is processed sequentially, updating the hidden state at each step.
  - Final hidden state represents the entire sequence.

- **LSTM:**
  - **Description:** Includes mechanisms to store and retrieve information over long periods.
  - **Example:**
    - Sentence: "The long-term dependencies in this sentence are handled well by LSTMs."
    - LSTM updates its cell state to remember or forget information as needed.

- **GRU:**
  - **Description:** Simplified version of LSTMs with fewer gates.
  - **Example:**
    - Similar usage as LSTMs, but with a more streamlined architecture.

### 6. **Convolutional Neural Networks (CNNs) for Text**
- **Description:** Uses convolutional layers to capture local patterns in text, often used for sentence and document classification.
- **Example:**
  - Text: "NLP with CNNs is effective."
  - Convolutional filters slide over the text to capture n-gram patterns.
  - Pooling layers reduce dimensionality, followed by fully connected layers for classification.

### 7. **Transformers**
- **Description:** Uses self-attention mechanisms to process sequences in parallel, capturing long-range dependencies more effectively.
- **Example Models:**
  - **BERT:**
    - **Description:** Pre-trained using masked language modeling and next sentence prediction, captures bidirectional context.
    - Example:
      - Sentence: "The quick brown fox jumps over the lazy dog."
      - BERT provides contextual embeddings for each word.
  - **GPT:**
    - **Description:** Pre-trained with a unidirectional approach, primarily used for text generation.
    - Example:
      - Given a prompt: "Once upon a time,"
      - GPT generates a continuation: "there was a brave knight."
  - **RoBERTa:** A robustly optimized version of BERT.
  - **T5 (Text-To-Text Transfer Transformer):** Converts all NLP tasks into a text-to-text format.
    - Example:
      - Task: "Translate English to French: The book is on the table."
      - Output: "Le livre est sur la table."

### 8. **Character-Level Embeddings**
- **Description:** Processes text at the character level, capturing subword information and handling out-of-vocabulary words.
- **Example:**
  - Text: "hello"
  - Character embeddings for ['h', 'e', 'l', 'l', 'o'] are processed by a model to understand the word.

### 9. **Byte-Pair Encoding (BPE)**
- **Description:** A subword tokenization technique that merges the most frequent pairs of characters or subwords iteratively.
- **Example:**
  - Word: "lower"
  - Initial pairs: ['l', 'o', 'w', 'e', 'r']
  - BPE merges: ['low', 'er']
  - Encoded: ['low', 'er']

### 10. **SentencePiece**
- **Description:** An unsupervised text tokenizer and detokenizer that handles subword units, commonly used with models like T5.
- **Example:**
  - Text: "This is a test."
  - Tokenization: ['▁This', '▁is', '▁a', '▁test', '.']
  - Subword tokens are prefixed with a special marker (▁).

These encoding techniques and models form the basis of many modern NLP applications, each with its strengths and suitable use cases depending on the nature of the text data and the specific task at hand.


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# print(encode("Anas Nouri")
print(decode(encode("ANAS NOURI")))
print(encode("ANAS NOURI"))
# print(decode([18, 47, 56, 57, 58]))

ANAS NOURI
[13, 26, 13, 31, 1, 26, 27, 33, 30, 21]


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")
# x,y

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [11]:
# print(len(data)) => 1115394

In [12]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")
# ix = torch.randint(len(data) - block_size, (batch_size,))
# ix

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [13]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [14]:
print(decode([44, 53, 56,  1, 58, 46, 39, 58]))

for that


In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            probabilities = F.softmax(logits, dim=-1)
            print(probabilities)
            targets = targets.view(B*T)
            print(targets)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


tensor([[0.0021, 0.0085, 0.0280,  ..., 0.0051, 0.0048, 0.0201],
        [0.0095, 0.0063, 0.0032,  ..., 0.0035, 0.0026, 0.0026],
        [0.0118, 0.0049, 0.0025,  ..., 0.0341, 0.0074, 0.0015],
        ...,
        [0.0007, 0.0030, 0.0464,  ..., 0.0045, 0.0153, 0.0035],
        [0.0172, 0.0090, 0.0088,  ..., 0.0022, 0.0012, 0.0587],
        [0.0045, 0.0212, 0.0017,  ..., 0.0950, 0.0041, 0.0069]],
       grad_fn=<SoftmaxBackward0>)
tensor([43, 58,  5, 57,  1, 46, 43, 39, 53, 56,  1, 58, 46, 39, 58,  1, 58,  1,
        58, 46, 39, 58,  1, 46, 17, 27, 10,  0, 21,  1, 54, 39])
torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [16]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [17]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


tensor([[0.0095, 0.0063, 0.0032,  ..., 0.0035, 0.0026, 0.0026],
        [0.0051, 0.0114, 0.0253,  ..., 0.0084, 0.0190, 0.0425],
        [0.0051, 0.0114, 0.0253,  ..., 0.0084, 0.0190, 0.0425],
        ...,
        [0.0172, 0.0090, 0.0088,  ..., 0.0022, 0.0012, 0.0587],
        [0.0392, 0.0142, 0.0010,  ..., 0.0195, 0.0134, 0.0108],
        [0.0075, 0.0049, 0.0095,  ..., 0.0036, 0.0017, 0.0043]],
       grad_fn=<SoftmaxBackward0>)
tensor([57, 57,  1, 58, 53,  1, 40, 43, 58, 46, 43, 56,  1, 39, 45, 43, 46, 39,
        58,  1, 61, 39, 63,  1, 39, 50, 50,  1, 40, 43,  1, 46, 43, 53,  6,  1,
        46, 47, 57, 58, 47, 64, 43, 52, 10,  0, 20, 43, 47, 53, 52,  0, 21, 57,
         1, 44, 57, 46, 43,  1, 61, 53, 59, 50,  0,  0, 23, 21, 26, 19,  1, 24,
        61, 46, 43, 56, 43,  1, 58, 46, 47, 57,  1, 46, 43, 39, 56, 58, 43, 39,
        56, 58, 43, 42,  1, 41, 18, 53, 59, 56, 58, 46,  1, 25, 15, 53, 51, 51,
        39, 52, 42,  1, 59, 57, 43,  6,  0, 32, 53,  1, 59, 56,  8,  0,  0, 25,
       

In [18]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


xiKi-RJ:CgqVuUa!U?qMH.uk!sCuMXvv!CJFfx;LgRyJknOEti.?I&-gPlLyulId?XlaInQ'q,lT$
3Q&sGlvHQ?mqSq-eON
x?SP fUAfCAuCX:bOlgiRQWN:Mphaw
tRLKuYXEaAXxrcq-gCUzeh3w!AcyaylgYWjmJM?Uzw:inaY,:C&OECW:vmGGJAn3onAuMgia!ms$Vb q-gCOcPcUhOnxJGUGSPJWT:.?ujmJFoiNL&A'DxY,prZ?qdT;hoo'dHooXXlxf'WkHK&u3Q?rqUi.kz;?Yx?C&u3Qbfzxlyh'Vl:zyxjKXgC?
lv'QKFiBeviNxO'm!Upm$srm&TqViqiBD3HBP!juEOpmZJyF$Fwfy!PlvWPFC
&WDdP!Ko,px
x
tREOE;AJ.BeXkylOVD3KHp$e?nD,.SFbWWI'ubcL!q-tU;aXmJ&uGXHxJXI&Z!gHRpajj;l.
pTErIBjx;JKIgoCnLGXrJSP!AU-AcbczR?


## The mathematical trick in self-attention

In [19]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [20]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [21]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)
xbow[1]

tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0.0020],
        [ 0.0712, -0.1128],
        [ 0.2527,  0.2149]])

In [22]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)
xbow2[1]

tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0.0020],
        [ 0.0712, -0.1128],
        [ 0.2527,  0.2149]])

In [23]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [24]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))# if we delete this line of code : we are called this : encoder -blocks talk to  each other
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape,wei.shape ,v.shape

(torch.Size([4, 8, 16]), torch.Size([4, 8, 8]), torch.Size([4, 8, 16]))

In [25]:

out[0]

tensor([[-0.1571,  0.8801,  0.1615, -0.7824, -0.1429,  0.7468,  0.1007, -0.5239,
         -0.8873,  0.1907,  0.1762, -0.5943, -0.4812, -0.4860,  0.2862,  0.5710],
        [ 0.6764, -0.5477, -0.2478,  0.3143, -0.1280, -0.2952, -0.4296, -0.1089,
         -0.0493,  0.7268,  0.7130, -0.1164,  0.3266,  0.3431, -0.0710,  1.2716],
        [ 0.4823, -0.1069, -0.4055,  0.1770,  0.1581, -0.1697,  0.0162,  0.0215,
         -0.2490, -0.3773,  0.2787,  0.1629, -0.2895, -0.0676, -0.1416,  1.2194],
        [ 0.1971,  0.2856, -0.1303, -0.2655,  0.0668,  0.1954,  0.0281, -0.2451,
         -0.4647,  0.0693,  0.1528, -0.2032, -0.2479, -0.1621,  0.1947,  0.7678],
        [ 0.2510,  0.7346,  0.5939,  0.2516,  0.2606,  0.7582,  0.5595,  0.3539,
         -0.5934, -1.0807, -0.3111, -0.2781, -0.9054,  0.1318, -0.1382,  0.6371],
        [ 0.3428,  0.4960,  0.4725,  0.3028,  0.1844,  0.5814,  0.3824,  0.2952,
         -0.4897, -0.7705, -0.1172, -0.2541, -0.6892,  0.1979, -0.1513,  0.7666],
        [ 0.1866, -0.0

In [26]:
key , query

(Linear(in_features=32, out_features=16, bias=False),
 Linear(in_features=32, out_features=16, bias=False))

In [27]:
k

tensor([[[ 1.1965e-01, -3.0127e-01,  3.6293e-01,  1.1771e+00,  1.1385e+00,
          -2.5543e-01,  1.4537e-01, -2.9437e-01, -7.0201e-01, -1.0308e+00,
           7.4357e-01, -8.0984e-01, -6.6687e-01,  9.1233e-02, -6.0747e-03,
           1.9833e-01],
         [-5.4229e-01, -5.5581e-01, -7.6131e-02,  1.2929e+00,  8.6535e-01,
          -1.1998e+00,  3.8781e-01,  1.9389e-01,  7.0235e-01, -8.2251e-01,
           2.3484e-01, -8.4995e-01, -3.8126e-01, -2.9906e-01,  1.0242e-02,
          -5.5449e-01],
         [-3.7359e-01, -4.6781e-01, -2.1560e-01, -8.0344e-01, -3.7153e-01,
          -5.4427e-01, -9.1455e-01, -5.5926e-02, -3.2903e-01, -2.1023e-01,
           1.1665e-01, -1.7978e-01, -2.8196e-01, -3.3204e-01, -4.5963e-01,
          -1.3255e-01],
         [-3.1463e-01,  8.4460e-02, -1.2351e-01, -7.0577e-01, -1.8022e-01,
           5.4922e-01, -8.9805e-01, -4.9384e-01,  6.7907e-01,  8.8270e-01,
           4.9109e-01,  5.1903e-01,  9.0109e-01,  9.1255e-02, -1.9332e-01,
          -6.7704e-01],
    

In [28]:
A = k.transpose(-2,-1)
A.squeeze(dim =1)

tensor([[[ 1.1965e-01, -5.4229e-01, -3.7359e-01, -3.1463e-01,  2.3940e-02,
          -2.3618e-01, -7.9413e-01,  1.6513e-01],
         [-3.0127e-01, -5.5581e-01, -4.6781e-01,  8.4460e-02,  9.9822e-02,
          -7.8730e-01, -1.6598e-01, -1.5990e-01],
         [ 3.6293e-01, -7.6131e-02, -2.1560e-01, -1.2351e-01, -1.8709e-01,
          -3.8019e-01, -2.8096e-01, -5.7168e-01],
         [ 1.1771e+00,  1.2929e+00, -8.0344e-01, -7.0577e-01, -8.5960e-02,
           5.8150e-01, -1.0208e-01, -3.9571e-01],
         [ 1.1385e+00,  8.6535e-01, -3.7153e-01, -1.8022e-01, -4.8815e-01,
          -3.7222e-01, -7.3521e-01,  3.9301e-01],
         [-2.5543e-01, -1.1998e+00, -5.4427e-01,  5.4922e-01, -1.6765e+00,
           1.2405e+00, -7.5183e-01, -8.5665e-01],
         [ 1.4537e-01,  3.8781e-01, -9.1455e-01, -8.9805e-01,  2.4126e-01,
          -7.0045e-01, -1.2759e-01,  3.3900e-01],
         [-2.9437e-01,  1.9389e-01, -5.5926e-02, -4.9384e-01,  7.3606e-01,
          -1.4917e+00, -5.1134e-03, -7.9771e-01],


In [29]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries these nodes (  hers means tokens  are slef-attending). In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [30]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [31]:
k.var()

tensor(1.0449)

In [32]:
q.var()

tensor(1.0700)

In [33]:
wei.var()

tensor(1.0918)

it 's really important  especially  at intialization that  Wei be fairly diffusee  so  in our case  here we sort  of lucked `*out*`  here and `*wei*` had a fairly diffuse numbers here
# so the probelm : if softmax  weights take on very positive and very negative inside it (  means here the variance is very high)=> softmax will be  actually convergen twords one hot  vecotrs

In [34]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)# as you see here we  are applying softmax to tenser of values that are very close to zero then we get we're going to a diffuse thing out softmax

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot (we take the same vector  like previous vector and we start sharpening it making it bigger by multiplying these numbers by eight for example  you see that softmax  will start to sharpen and  fact it will sharpen  towards the max  so  it will sharpen towards whatever numberrs here(0.800)    )

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

because we don't want these values to be too  extreme especially in intialization otherwise softmax will be way to peaky (converge towards max) and you're basically aggregating information from like  a single  node ( token here the token that has the big probel in softmax like ( 0.800)) so every other node ( token ) just aggregate informations  from  a single  => this is not what we want especially its intialization  => the solution is `*scaling *` is used  just  to control the variance at initialization

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
import torch
torch.arange(T)

tensor([0, 1, 2, 3, 4, 5, 6, 7])

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T) in general (C= head_size  but in this case head_size  =  n_emb)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]#we can never have more than coming in because  if idex is more than block size then our position emedding table is going to run out of scope because it has only embedding for up to block size
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5090, val loss 2.5058
step 300: train loss 2.4198, val loss 2.4340
step 400: train loss 2.3503, val loss 2.3567
step 500: train loss 2.2970, val loss 2.3136
step 600: train loss 2.2410, val loss 2.2506
step 700: train loss 2.2062, val loss 2.2198
step 800: train loss 2.1638, val loss 2.1871
step 900: train loss 2.1232, val loss 2.1494
step 1000: train loss 2.1020, val loss 2.1293
step 1100: train loss 2.0704, val loss 2.1196
step 1200: train loss 2.0382, val loss 2.0798
step 1300: train loss 2.0249, val loss 2.0640
step 1400: train loss 1.9922, val loss 2.0354
step 1500: train loss 1.9707, val loss 2.0308
step 1600: train loss 1.9614, val loss 2.0474
step 1700: train loss 1.9393, val loss 2.0130
step 1800: train loss 1.9070, val loss 1.9943
step 1900: train loss 1.9057, val loss 1.9871
step 2000: train loss 1.8834, val loss 1.9954
step 2100: train loss 1.